In [80]:
%autoreload 2

In [81]:
from argparse import Namespace
from collections import defaultdict
import copy
from datetime import datetime
import difflib
import duckdb
from functools import reduce
import glob
import gzip
import itertools
import os
import pickle
import sys
import typing

from IPython.display import display, Markdown, HTML  # type: ignore
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import tatsu, tatsu.ast
import tqdm.notebook as tqdmn


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../reward-machine'))

import compile_predicate_statistics
import compile_predicate_statistics_split_args
from compile_predicate_statistics_split_args import *
from config import SPECIFIC_NAMED_OBJECTS_BY_ROOM
import config
from utils import get_object_assignments

import ast_printer
import ast_parser

In [96]:
cache_dir = compile_predicate_statistics.get_project_dir() + '/reward-machine/caches'

bitstings_df_path = os.path.join(cache_dir, 'predicate_statistics_bitstring_intervals_028b3733.pkl.gz')
bitstrings_df = pd.read_pickle(bitstings_df_path)
with gzip.open(os.path.join(cache_dir, 'trace_lengths_028b3733.pkl'), 'rb') as f:
    trace_lengths_and_domains = pickle.load(f)


# regular_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics.pkl'))
# split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_028b3733.pkl.gz'))
# split_args_df = split_args_df[split_args_df['predicate'] != 'same_type']
# print(split_args_df.shape)
# # split_args_df = pd.read_pickle(os.path.join(cache_dir, 'predicate_statistics_4d5dd602.pkl.gz'))

# # stats = compile_predicate_statistics.CommonSensePredicateStatistics(cache_dir)
# split_args_stats = compile_predicate_statistics_split_args.CommonSensePredicateStatisticsSplitArgs(
#     # cache_dir, compile_predicate_statistics_split_args.CURRENT_TEST_TRACE_NAMES, overwrite=False
#     )

In [90]:
l = 'blue_cube_block'.split('_')
l[1:] + l[:1] 

['cube', 'block', 'blue']

In [97]:
RENAMED_TYPES = """blue_cube_block
tan_cube_block
yellow_cube_block
blue_pyramid_block
red_pyramid_block
yellow_pyramid_block
blue_dodgeball
red_dodgeball
pink_dodgeball
green_golfball
green_triangular_ramp""".split('\n')

class DefaultValueDict(dict):
    def __init__(self, *args, **kawags):
        super().__init__(*args, **kawags)

    def __missing__(self, key):
        return key
    
arg_type_mapping = DefaultValueDict()
for renamed_type in RENAMED_TYPES:
    sp = renamed_type.split('_')
    new_name = '_'.join(sp[1:] + sp[:1])
    arg_type_mapping[renamed_type] =  new_name



bitstrings_df.assign(arg_1_type=bitstrings_df.arg_1_type.map(arg_type_mapping), 
                     arg_2_type=bitstrings_df.arg_2_type.map(arg_type_mapping),).to_pickle(bitstings_df_path)

In [ ]:
bitstrings_df.replace(dict(domain=dict(few=0, medium=1, many=2))).to_pickle('predicate_statistics_modified_bitstring_intervals_028b3733.pkl.gz')

In [ ]:
split_args_stats.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()

In [ ]:
a = np.zeros(10000, dtype=np.uint8)
a[100:1000] = 1


In [ ]:
t = np.zeros(10, dtype=np.uint8)
t[1:4] = 1
b = t.tobytes()
d = {b[0]: '0', b[1]: '1'}

In [ ]:
''.join(map(lambda x: d[x], b))

In [ ]:
b = bytes([0, 1])
b

In [ ]:
MAX_LENGTH = split_args_stats.trace_lengths_and_domains_df.select(pl.col('trace_length').max()).item()
print(MAX_LENGTH)
b = bytes([0, 1])
BYTE_MAPPING = {b: str(i) for i, b in enumerate(b)}



def intervals_to_string(intervals, max_length: int = MAX_LENGTH):
    value = np.zeros(max_length, dtype=np.uint8)
    for interval in intervals:
        value[interval[0]:interval[1]] = 1

    return ''.join(map(lambda b: BYTE_MAPPING[b], value.tobytes()))
    # return np.array2string(value, separator='', threshold=max_length + 10)[1:-1].replace('\n ', '')


intervals = split_args_df.intervals.apply(intervals_to_string)

# small_split_args_df = small_split_args_df.assign()

In [ ]:
split_args_df = split_args_df.assign(string_intervals=intervals)
split_args_df.head()

In [3]:
import compile_predicate_statistics_full_database

stats = compile_predicate_statistics_full_database.CommonSensePredicateStatisticsFullDatabase(force_trace_names_hash='028b3733')

2023-08-22 17:47:56 - compile_predicate_statistics_full_database - INFO     - Loading data from files
2023-08-22 17:48:09 - compile_predicate_statistics_full_database - INFO     - Creating DuckDB table...
2023-08-22 17:48:11 - compile_predicate_statistics_full_database - INFO     - Loaded data, found 1708171 rows


In [4]:
from src.ast_utils import cached_load_and_parse_games_from_file
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='.'))

2023-08-22 17:48:11 - src.ast_utils - DEBUG    - Using cache folder: /Users/guydavidson/tmp/game_generation_cache
2023-08-22 17:48:13 - src.ast_utils - INFO     - Loading from cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz
2023-08-22 17:48:13 - src.ast_utils - INFO     - Finished loading cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz


In [76]:
test_game_str = """
(define (game test-game) (:domain many-objects-room-v1)
(:constraints (and
    (preference testPred
        (exists (?v1 - building ?v2 - flat_block)
            (at-end (and
                (in ?v2 ?v1)
            ))
        )
    )
))
(:scoring (+
    (* (count testPred) 1)
)))
""".strip()
# test_game_str = """
# (define (game test-game) (:domain many-objects-room-v1)
# (:constraints (and
#     (preference testPred
#         (exists (?v0 - block ?v1 - wall)
#             (at-end (and 
#                 (on ?v1 ?v0)
#                 (in ?v0 ?v1)
#             ))
#         )
#     )
# ))
# (:scoring (+
#     (* (count testPred) 1)
# )))
# """.strip()

test_ast = grammar_parser.parse(test_game_str)
test_pred = test_ast[3][1].preferences[0].definition.pref_body.body.exists_args.at_end_pred
print(ast_printer.ast_section_to_string(test_pred, ast_parser.PREFERENCES))

(and  (in ?v2 ?v1))


In [79]:
df = stats.filter(test_pred, {'?v1': ['pillow'], '?v2': ['hexagonal_bin']}, last_interval_bit_set=True)
# df = stats.filter(test_pred, {'?v1': ['building'], '?v2': ['flat_block'], '?v3': ['block']}, last_interval_bit_set=True)
df

,trace_id,domain,last_bit,?v2,?v1
0,9C0wMm4lzrJ5JeP0irIu-gameplay-attempt-1-rereco...,few,1,GarbageCan|+00.95|-00.03|-02.68,Pillow|-02.45|+00.66|+00.10
1,pOvjLn7nuSqcxgqhaNuv-preCreateGame-rerecorded,few,0,GarbageCan|+00.95|-00.03|-02.68,Pillow|-02.45|+00.66|+00.10
2,rRNy5QUGjCQzjIYuZvWJ-preCreateGame-rerecorded,few,0,GarbageCan|+00.95|-00.03|-02.68,Pillow|-02.45|+00.66|+00.10
3,tVL75ycOZHTAKgeqLs6H-preCreateGame-rerecorded,few,0,GarbageCan|+00.95|-00.03|-02.68,Pillow|-02.45|+00.66|+00.10


In [74]:
df[df.last_bit == 1]

,trace_id,domain,last_bit,?v2,?v3,?v1
1,GLPtcvJUaHkUYK7iEPRq-createGame-rerecorded,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,FlatRectBlock|+00.23|+01.66|-02.88,building_2
2,GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rereco...,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,BridgeBlock|+01.03|+01.11|-02.88,building_0
4,GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rereco...,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,FlatRectBlock|+00.23|+01.66|-02.88,building_0
5,GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rereco...,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,LongCylinderBlock|+00.31|+01.19|-02.89,building_0
8,GLPtcvJUaHkUYK7iEPRq-gameplay-attempt-1-rereco...,medium,1,FlatRectBlock|+00.24|+01.57|-02.89,BridgeBlock|+00.63|+01.10|-02.91,building_3
...,...,...,...,...,...,...
260,EB9bKJMBxNqVaRtrXDuO-createGame-rerecorded,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,CubeBlock|+00.70|+01.61|-02.91,building_0
261,EB9bKJMBxNqVaRtrXDuO-createGame-rerecorded,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,BridgeBlock|+01.03|+01.11|-02.88,building_0
264,EB9bKJMBxNqVaRtrXDuO-createGame-rerecorded,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,FlatRectBlock|+00.23|+01.66|-02.88,building_0
287,EB9bKJMBxNqVaRtrXDuO-createGame-rerecorded,medium,1,FlatRectBlock|+00.23|+01.66|-02.88,BridgeBlock|+00.63|+01.10|-02.91,building_0


In [ ]:
p = game_asts[1][4][1].preferences[0].definition.forall_pref.preferences.pref_body.body.then_funcs[0].seq_func.once_pred
print(p.keys())
ast_printer.ast_section_to_string(p, ast_parser.PREFERENCES)

In [ ]:
q = stats.filter(p, {"?b": ["ball"], "?t": ["hexagonal_bin"]})
print(q)

In [ ]:
duckdb.sql('PRAGMA force_index_join;')
duckdb.sql("PRAGMA explain_output='OPTIMIZED_ONLY';")

In [ ]:
# q = """
#     SELECT t0.trace_id, t0.domain, t0."?b", t1."door", t1."agent", (t0.intervals & t1.intervals) AS intervals
#     FROM (SELECT trace_id, domain, intervals, arg_1_id AS "?b" FROM data WHERE predicate='agent_holds' AND (arg_1_type IN ('beachball'::arg_type, 'basketball'::arg_type, 'dodgeball'::arg_type, 'golfball'::arg_type))) as t0
#     INNER JOIN (SELECT trace_id, domain, intervals, arg_1_id AS "door", arg_2_id AS "agent" FROM data WHERE predicate='adjacent' AND (arg_1_type='door') AND (arg_2_type='agent')) as t1
#     ON (t0.trace_id=t1.trace_id)
# """

q = """
    SELECT t0.trace_id, t0.domain, t0.arg_1_id AS "?b", t1.arg_1_id AS "door", t1.arg_2_id AS "agent", (t0.intervals & t1.intervals) AS intervals
    FROM data AS t0
    INNER JOIN data AS t1
    ON (t0.trace_id=t1.trace_id)
    WHERE t0.predicate='agent_holds' AND (t0.arg_1_type IN ('beachball'::arg_type, 'basketball'::arg_type, 'dodgeball'::arg_type, 'golfball'::arg_type)) AND
    t1.predicate='adjacent' AND (t1.arg_1_type='door') AND (t1.arg_2_type='agent')

"""

print(duckdb.sql(f"EXPLAIN ANALYZE ({q})").fetchone()[1])

In [ ]:
duckdb.sql("INSERT INTO domains VALUES ('few'), ('medium'), ('many')")

In [ ]:
df = duckdb.sql(q).fetchdf()

In [ ]:
duckdb.sql('CREATE INDEX idx_data_predicate ON data (predicate)')

In [ ]:
t = np.zeros(10, dtype=np.uint8)
t[1:4] = 1
b = t.tobytes()
d = {b[0]: '0', b[1]: '1'}

In [ ]:
df[df.trace_id == '1El1CmicSoKZKTLe8NpP-preCreateGame-rerecorded']

In [ ]:
bits_from_df = np.unpackbits(np.frombuffer(interval_from_df, dtype=np.uint8))
len(bits_from_df), bits_from_df

In [ ]:
results = duckdb.sql(q).fetchall()

for tup in results:
    sub_df = df[(tup[0] == df['trace_id']) & (tup[2] == df['?b']) & (tup[3] == df['?t'])]
    if len(sub_df) != 1:
        print(f'Error: {tup[:-1]}')
        print(len(sub_df))
        break

    expected_length = duckdb.sql(f"SELECT length from trace_length_and_domains WHERE trace_id='{tup[0]}'").fetchone()[0]

    bits_from_df = np.unpackbits(np.frombuffer(sub_df.intervals.item(), dtype=np.uint8))
    bits_from_db = np.fromiter(map(int, tup[-1]), dtype=np.uint8)

    if len(bits_from_db) != expected_length:
        print(f'Error: {tup[:-1]}')
        print(len(bits_from_db))
        print(expected_length)
        print(len(bits_from_df))
        break

    if not np.all(bits_from_df[-expected_length:] == bits_from_db):
        print(f'Error: {tup[:-1]}')
        print(np.where(bits_from_df[-expected_length:] != bits_from_db))


In [ ]:
tup[:-1]

In [ ]:
len(bits_from_df[-550:])

In [ ]:
bits_from_db = np.fromiter(map(int, t[-1]), dtype=np.uint8)

In [ ]:
bits_from_df = np.unpackbits(np.frombuffer(df[df.trace_id == t[0]].intervals.item(), dtype=np.uint8))

In [ ]:
print(len(bits_from_db), len(bits_from_df))

In [ ]:
1264 % 16

In [ ]:
print(len(bits_from_db), len(bits_from_df))
np.where(bits_from_db[:] != bits_from_df[14:])

In [ ]:
bits_from_db[500:540], bits_from_df[500:540]

In [ ]:
(bits_from_db[:min_length] != bits_from_df[:min_length])[:10]

## Gameplan from here

* For (setup, each preference):
* Inspect to make sure all predicates implemeneted in the cache, it doesn't use a once-measure (and probably also not a hold-while, at least at first), etc. 
    * otherwise default to the basic implementation
* If we're go to run on a particular thing:
    * If it's a setup, it's trivial
    * If it's a preference, check if it's an at-end or then
        * If it's an at-end, it's trivial (can probably fold this into the query by doing `get_bit(intervals, length - 1)` if we join on the trace lengths table
        * If it's a then, enumerate over the predicates of each modal, and query for them
            * For each one, fetch the df for the query, use the trace lengths to transform the intervals to the expected format
            * Add the index of the modal to the df
            * Enumerate through all trace ids and assignments, and for each assignment where we have all modals represented: 
                * Create the joint state interval
                * Iterate through the joint using the state machine logic
                * Count satisfactions
                * ...
                * Profit!

In [ ]:
import tatsu.ast
import tatsu.grammars
from ast_parser import ASTParser, SECTION_CONTEXT_KEY, VARIABLES_CONTEXT_KEY
from ast_utils import simplified_context_deepcopy, deepcopy_ast, ASTCopyType, replace_child


DEFAULT_UNSUPPORTED_RULES = [
    'function_comparison',
    'function_eval',
    'predicate_adjacent_side_3',
    'predicate_adjacent_side_4',
    'predicate_between',
    'predicate_faces',
    'predicate_is_setup_object',
    'predicate_opposite',
    'predicate_rug_color_under',
    'predicate_same_color',
    'predicate_same_object',
    'predicate_same_type',
    'super_predicate_exists',
    'super_predicate_forall',
    'once_measure',
    'while_hold',
]


def _pref_forall_pos_to_key(pos: int):
    return f'pref_forall_{pos}'


class MixedTraceFilterGameParser(ASTParser):
    unsupported_rules: typing.Set[str]

    def __init__(self, unsupported_rules: typing.Sequence[str] = DEFAULT_UNSUPPORTED_RULES):
        super().__init__()
        self.expected_keys = set()
        self.unsupported_rules = set(unsupported_rules)

    def __call__(self, ast, **kwargs):
        initial_call = 'inner_call' not in kwargs or not kwargs['inner_call']
        if initial_call:
            kwargs['inner_call'] = True
            kwargs['local_context'] = {'mapping': {VARIABLES_CONTEXT_KEY: {}}}
            kwargs['global_context'] = {}
            self.expected_keys = set()
            self.unsupported_keys = set()
            # self.traces_by_preference_or_section = {}
            # self.preferences_or_sections_with_implemented_predicates = set()
            # self.predicate_strings_by_preference_or_section = defaultdict(set)
            # self.not_implemented_predicate_counts = defaultdict(int)

        retval = super().__call__(ast, **kwargs)

        if initial_call:
            return self.unsupported_keys, self.expected_keys
        else:
            return retval

    def _current_ast_to_contexts_hook(self, ast: tatsu.ast.AST, kwargs: typing.Dict[str, typing.Any]):
        rule = typing.cast(str, ast.parseinfo.rule)  # type: ignore

        if rule == 'pref_forall':
            kwargs['local_context']['current_pref_forall_index'] = ast.parseinfo.pos

        if rule == 'preference':
            kwargs['local_context']['current_preference_name'] = ast.pref_name

    def _handle_ast(self, ast: tatsu.ast.AST, **kwargs):
        self._current_ast_to_contexts(ast, **kwargs)
        kwargs['local_context']['mapping'] = ast_parser.update_context_variables(ast, kwargs['local_context']['mapping'])

        current_key = None
        if SECTION_CONTEXT_KEY in kwargs and kwargs[SECTION_CONTEXT_KEY] == ast_parser.SETUP:
            current_key = kwargs[SECTION_CONTEXT_KEY]
        elif 'current_pref_forall_index' in kwargs['local_context']:
            current_key =_pref_forall_pos_to_key(kwargs['local_context']['current_pref_forall_index'])
        elif 'current_preference_name' in kwargs['local_context']:
            current_key = kwargs['local_context']['current_preference_name']
        
        if current_key is not None:
            self.expected_keys.add(current_key)

            if ast.parseinfo.rule in self.unsupported_rules:
                self.unsupported_keys.add(current_key)

        for key in ast:
            if key != 'parseinfo':
                child_kwargs = simplified_context_deepcopy(kwargs)
                retval = self(ast[key], **child_kwargs)
                self._update_contexts_from_retval(kwargs, retval)
            
            


In [ ]:
game_parser = MixedTraceFilterGameParser()
for ast in game_asts:
    unsupported, expected = game_parser(ast)
    supported = expected - unsupported
    print(f'Game {ast[1].game_name} has supported keys: {list(supported)} and unsupported keys: {list(unsupported)}')

In [ ]:



DUMMY_PREFERENCE_GAME = """(define (game dummy-preference-game) (:domain many-objects-room-v1)
(:constraints (and
    (preference dummyPreference
            (at-end (game-over))
    )
))
(:scoring (count dummyPreference)
))
"""


class ASTTraceFilterSplitter(ast_parser.ASTParser):
    keep_keys: typing.Set[str]
    remove_keys: typing.Set[str]
    should_insert_dummy_preference: bool
    
    def __init__(self, grammar_parser: tatsu.grammars.Grammar):
        self.grammar_parser = grammar_parser

    def __call__(self, ast, **kwargs):
        initial_call = 'inner_call' not in kwargs or not kwargs['inner_call']
        if initial_call:
            kwargs['inner_call'] = True
            
            if 'remove_keys' not in kwargs:
                raise ValueError('remove_keys must be specified')
            self.remove_keys = kwargs['remove_keys']

            if len(self.remove_keys) == 0:
                raise ValueError('remove_keys must be non-empty')

            if 'keep_keys' not in kwargs:
                raise ValueError('keep_keys must be specified')
            self.keep_keys = kwargs['keep_keys']

            if len(self.keep_keys) == 0:
                raise ValueError('keep_keys must be non-empty')

            ast = deepcopy_ast(ast)

            # Handle the setup right here and now, if we're removing it
            if ast_parser.SETUP in self.remove_keys:
                ast = (*ast[:3], *ast[4:])
                # If the only thin we're removing is the setup, we're done
                if len(self.remove_keys) == 1:
                    return ast

            # check if we're only keeping the setup and inserting a dummy preference, because if so, we're done
            if len(self.keep_keys) == 1 and ast_parser.SETUP in self.keep_keys:
                dummy_preference_game = self.grammar_parser.parse(DUMMY_PREFERENCE_GAME)
                return (*ast[:4], dummy_preference_game[3], *ast[4:])

        super().__call__(ast, **kwargs)

        if initial_call:
            return ast
        
    def _handle_ast(self, ast: tatsu.ast.AST, **kwargs):
        rule = ast.parseinfo.rule

        if rule == 'preferences':
            if isinstance(ast.preferences, tatsu.ast.AST):
                raise ValueError(f'If removing a single preference, the initial call should handle it, so this should never occur')
            
            new_children = typing.cast(typing.List[tatsu.ast.AST], deepcopy_ast(ast.preferences, ASTCopyType.NODE))
            indices_to_remove = []
            for i, child in enumerate(new_children):
                if child.parseinfo.rule == 'preference' and child.pref_name in self.remove_keys:
                    print(f'Removing preference {child.pref_name}')
                    indices_to_remove.append(i)
                elif child.parseinfo.rule == 'pref_forall' and _pref_forall_pos_to_key(child.parseinfo.pos) in self.remove_keys:
                    print(f'Removing pref_forall {_pref_forall_pos_to_key(child.parseinfo.pos)}')
                    indices_to_remove.append(i)

            for i in reversed(indices_to_remove):
                new_children.remove(new_children[i])

            replace_child(ast, 'preferences', new_children)

        else:
            for key in ast:
                if key != 'parseinfo':
                    self(ast[key], **kwargs)

            
        


In [ ]:
game_parser = MixedTraceFilterGameParser()
game_splitter = ASTTraceFilterSplitter(grammar_parser)  # type: ignore
ast = game_asts[0]
unsupported, expected = game_parser(ast)
supported = expected - unsupported

if len(supported) > 0 and len(unsupported) > 0:
    print(f'Game {ast[1].game_name} has supported keys: {list(supported)} and unsupported keys: {list(unsupported)}')
    supported_only = game_splitter(ast, keep_keys=supported, remove_keys=unsupported)
    unsupported_only = game_splitter(ast, keep_keys=unsupported, remove_keys=supported)

    print('=' * 80)
    print(ast_printer.ast_to_string(supported_only, '\n'))
    print('=' * 80)
    print(ast_printer.ast_to_string(unsupported_only, '\n'))

In [ ]:
from itertools import chain
categorical_type = pd.api.types.CategoricalDtype(sorted(chain.from_iterable(duckdb.sql("SELECT enum_range(NULL::trace_id)").fetchone())), ordered=True)

In [ ]:
trace_id_to_length_df = duckdb.sql('SELECT * FROM trace_length_and_domains').fetchdf()
trace_id_to_length_df.drop(columns=['domain'], inplace=True)
trace_id_to_length_df.rename(columns=dict(length='trace_length'), inplace=True)
# trace_id_to_length_df.astype(dict(trace_id=categorical_type), copy=False)
trace_id_to_length_df.head()

In [ ]:
def _df_intervals_to_array(row):
    return np.unpackbits(np.frombuffer(row['intervals'], dtype=np.uint8))[-row['trace_length']:]


In [ ]:
df.astype(dict(trace_id=categorical_type), copy=False)

In [ ]:
# df.join(trace_id_to_length_df, on=['trace_id'], how='outer', rsuffix='_r')
merged_df = df.merge(trace_id_to_length_df, on=['trace_id'], how='left')
merged_df

In [ ]:
assigned_df = df.assign(intervals=merged_df.apply(_df_intervals_to_array, axis=1))
assigned_df

In [ ]:
# assigned_df.groupby('trace_id', as_index=True, observed=False).intervals.transform(lambda l: list(np.logical_or.reduce(*l)))

series_1 = assigned_df.groupby('trace_id', as_index=True, observed=True).intervals.agg(lambda x: reduce(np.logical_or, x.values).astype(bool))

In [ ]:
series_2 = series_1.iloc[:50]

In [ ]:
merged = pd.merge(series_1, series_2, left_index=True, right_index=True, how='inner')
s = merged.agg(lambda row: np.logical_and(row['intervals_x'], row['intervals_y']), axis=1)

In [ ]:
s.agg(np.logical_or.reduce).sum()